In [9]:
import mysql.connector
import pandas as pd
import bcrypt

def establish_sql_connection():
    # note: for testing purposes, run on localhost
    db = mysql.connector.connect(
        host = "localhost",
        port = 1001,
        user = "root",
        password = "dsa3101",
        database = "secdb"
        )
    cursor = db.cursor()
    # uncomment below for dockerised usage
    # db = mysql.connector.connect(
    #     host = "database",
    #     user = "root",
    #     password = "dsa3101",
    #     database = "secdb"
    #     )
    return db, cursor

In [2]:
# add users
def add_user_roles(filepath):

    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## read users from path and change to QUERY format
        file = open(filepath, 'r')
        values = file.read().split("\n")
        values = list(map(lambda x: f'(\'{x}\')',values))
        values = ", ".join(values)
        query = f'INSERT INTO User_roles(role) VALUES {values};'

        cursor.execute(query)
        db.commit()
        print(f"Number of rows affected: {cursor.rowcount}")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")


In [3]:
add_user_roles('User_roles.txt')

db established: True
Number of rows affected: 2


In [100]:
def add_user(username, password, role, email=None):
    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## generate hash and salt
        salt = bcrypt.gensalt()
        hash = bcrypt.hashpw(password.encode('utf-8'), salt)

        ## get role id
        query = f'SELECT id FROM User_roles \
                                 where role = \'{role}\';'
        cursor.execute(query)
        role_id=cursor.fetchall()[0][0]

        ## insert query
        query = f"\
            INSERT INTO Users\
            (email, username, role_id, salt, hash)\
            VALUES (\'{email}\', \'{username}\', {role_id}, \'{salt.decode('utf-8')}\', \'{hash.decode('utf-8')}\')"
        cursor.execute(query)
        db.commit()

    except mysql.connector.Error as err:
        print(f"Database error: {err}")

    finally:
        cursor.close()
        db.close()



In [101]:
add_user('sec1','sec1','security','sec1@nus.edu.sg')

db = establish_sql_connection()
cursor = db.cursor()
query = "SELECT * from Users"
cursor.execute(query)
Users = cursor.fetchall()
print(f'Users: {Users}')
cursor.close()
db.close()

db established: True
Users: [(1, 'sec1', 'sec1@nus.edu.sg', 1, '$2b$12$yefe8Q4bk54xKpBPItKva.', '$2b$12$yefe8Q4bk54xKpBPItKva.WBg5Z9bPcmii7SLmj8k0SO1jokAlpr.')]


In [109]:
def authenticate(user,password):
    try:
        db = establish_sql_connection()
        cursor = db.cursor()
        query = f"SELECT hash from Users where username = \'{user}\'"
        cursor.execute(query)
        hash_stored = cursor.fetchall()

        ## Check username
        if len(hash_stored) == 0:
            print("User not found")
            return False
        
        ## Pull shash salt
        hash_stored = hash_stored[0][0]
        query = f"SELECT salt from Users where username = \'{user}\'"
        cursor.execute(query)
        salt = cursor.fetchall()[0][0]
        hash_entered = bcrypt.hashpw(password.encode('utf-8'),salt.encode('utf-8'))
    
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    
    finally:
        cursor.close()
        db.close()

    ## Check results
    authenticated = bcrypt.checkpw(password.encode('utf-8'),hash_stored.encode('utf-8'))
    
    return authenticated




In [113]:
authenticate("sec1","sec1")
authenticate("sec1","sec2")
authenticate("sec2","sec2")

User not found


False

In [ ]:
data = pd.read_csv("back-end/data/data_except_priority&status.csv")
df = pd.DataFrame(data)


In [21]:
def add_incident_types(filepath):
    try: 
        ## establish connection
        db, cursor = establish_sql_connection()
        if db is None:
            print("add_incident_types | Failed to establish a database connection.")
            return
        print(f'add_incident_types | db established: {db.is_connected()}')

        ## read incident types from path
        with open(filepath, 'r') as file:
            lines = file.readlines()
            values = [tuple(line.strip().split(", ")) for line in lines]

        # Format values for the query
        formatted_values = ", ".join([f"('{type}', '{priority}')" for type, priority in values])

        query = f'INSERT INTO Incident_types(type, default_priority) VALUES {formatted_values};'

        cursor.execute(query)
        db.commit()
        print(f"add_incident_types | Incident types added: {cursor.rowcount}")

        cursor.close()
        db.close()

    except mysql.connector.Error as err:
        print(f"add_incident_types | database error: {err}")


In [22]:
add_incident_types('Incident_types.txt')

add_incident_types | db established: True
('LOST AND FOUND', 'Normal'), ('DAMAGED PROPERTY', 'High'), ('STOLEN ITEMS', 'Normal'), ('EMERGENCY INCIDENTS', 'High'), ('SEXUAL INCIDENTS', 'High')
add_incident_types | Incident types added: 5
